In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from rake_nltk import Rake


In [2]:
df = pd.read_csv('medium.csv')

/var/folders/1r/_g41q09n5dl_vr4brp9p_w580000gn/T/ipykernel_18330/261512805.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('medium.csv')


In [3]:
df = df.dropna(subset=['Title', 'Subtitle'])
df

,Unnamed: 0,Title,Subtitle,Image,Author,Publication,Year,Month,Day,Reading_Time,...,Tag_travel,Tag_trump,Tag_ux,Tag_venture-capital,Tag_web-design,Tag_web-development,Tag_women,Tag_wordpress,Tag_work,Tag_writing
0,2,Online Animation: Mixamo vs Norah AI,Online animations tools provide game designers...,1,Emma Laurent,NaN,2017,8,1,5,...,0,0,0,0,0,0,0,0,0,0
2,11,"Futures of AI, Friendly AI?",From Physics of the Future: How Science Will S...,1,Z,NaN,2017,8,1,3,...,0,0,0,0,0,0,0,0,0,0
3,12,The Value Generated through AI led Mindfulness,A true product co-created by the Millennials,1,Emiko Sawaguchi,MILLENNIALSTIMES,2017,8,1,5,...,0,0,0,0,0,0,0,0,0,0
4,17,"Os Melhores Links de JulhoCmeras Vintage, Goog...",mas pera a?,1,Giovani Ferreira,NEW ORDER,2017,8,1,3,...,0,0,0,0,0,0,0,0,0,0
5,18,The Future of AI: Redefining How We Imagine,Original article posted on FICO blogs,1,Syed Sadat Nazrul,Towards Data Science,2017,8,1,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391026,437502,", , !",Cryptocurrency: 1 ?,1,Zolbayar Zolo,"Zolbayar Bayarsaikhan Zolo Coder, Blogger",2017,9,30,6,...,0,0,0,0,0,0,0,0,0,0
1391027,437503,,AI Deep Learning : 2 -,1,Zolbayar Zolo,"Zolbayar Bayarsaikhan Zolo Coder, Blogger",2017,9,20,7,...,0,0,0,0,0,0,0,0,0,0
1391028,437504,(Shallow Neural Network),AI Deep Learning : 3,1,Zolbayar Zolo,"Zolbayar Bayarsaikhan Zolo Coder, Blogger",2018,1,18,5,...,0,0,0,0,0,0,0,0,0,0
1391030,437506,ATTN students: A handy tool to help you write ...,https://writewellapp.com/,0,Michael Musgrove,ZXOXZ,2017,10,15,1,...,0,0,0,0,0,0,0,0,0,1


In [4]:
df = df[df['Title'].apply(lambda x: len(x) > 5)]
df = df[df['Subtitle'].apply(lambda x: len(x) > 5)]
df['Text'] = df['Title'] + ' ' + df['Subtitle']


rows_to_drop = int(0.97 * len(df))

# Randomly select rows to drop
rows_to_keep = df.sample(n=len(df) - rows_to_drop, random_state=42)

# Update the DataFrame to keep only the randomly selected rows
df = rows_to_keep

# df = df.head(10000)

In [5]:
ind = 1
tag_weights = {}
inverse_tag_weights = {}

for elem in df:
    print(elem)
    if 'Tag' in elem and elem not in tag_weights:
        tag_weights[elem] = ind
        inverse_tag_weights[ind] = elem
        ind += 1
        
tag_weights

Unnamed: 0
Title
Subtitle
Image
Author
Publication
Year
Month
Day
Reading_Time
Claps
url
Author_url
Tag_ai
Tag_android
Tag_apple
Tag_architecture
Tag_art
Tag_artificial-intelligence
Tag_big-data
Tag_bitcoin
Tag_blacklivesmatter
Tag_blockchain
Tag_blog
Tag_blogging
Tag_books
Tag_branding
Tag_business
Tag_college
Tag_computer-science
Tag_creativity
Tag_cryptocurrency
Tag_culture
Tag_data
Tag_data-science
Tag_data-visualization
Tag_deep-learning
Tag_design
Tag_dogs
Tag_donald-trump
Tag_economics
Tag_education
Tag_energy
Tag_entrepreneurship
Tag_environment
Tag_ethereum
Tag_feminism
Tag_fiction
Tag_food
Tag_football
Tag_google
Tag_government
Tag_happiness
Tag_health
Tag_history
Tag_humor
Tag_inspiration
Tag_investing
Tag_ios
Tag_javascript
Tag_jobs
Tag_journalism
Tag_leadership
Tag_life
Tag_life-lessons
Tag_love
Tag_machine-learning
Tag_marketing
Tag_medium
Tag_mobile
Tag_motivation
Tag_movies
Tag_music
Tag_nba
Tag_news
Tag_nutrition
Tag_parenting
Tag_personal-development
Tag_photography
T

{'Tag_ai': 1,
 'Tag_android': 2,
 'Tag_apple': 3,
 'Tag_architecture': 4,
 'Tag_art': 5,
 'Tag_artificial-intelligence': 6,
 'Tag_big-data': 7,
 'Tag_bitcoin': 8,
 'Tag_blacklivesmatter': 9,
 'Tag_blockchain': 10,
 'Tag_blog': 11,
 'Tag_blogging': 12,
 'Tag_books': 13,
 'Tag_branding': 14,
 'Tag_business': 15,
 'Tag_college': 16,
 'Tag_computer-science': 17,
 'Tag_creativity': 18,
 'Tag_cryptocurrency': 19,
 'Tag_culture': 20,
 'Tag_data': 21,
 'Tag_data-science': 22,
 'Tag_data-visualization': 23,
 'Tag_deep-learning': 24,
 'Tag_design': 25,
 'Tag_dogs': 26,
 'Tag_donald-trump': 27,
 'Tag_economics': 28,
 'Tag_education': 29,
 'Tag_energy': 30,
 'Tag_entrepreneurship': 31,
 'Tag_environment': 32,
 'Tag_ethereum': 33,
 'Tag_feminism': 34,
 'Tag_fiction': 35,
 'Tag_food': 36,
 'Tag_football': 37,
 'Tag_google': 38,
 'Tag_government': 39,
 'Tag_happiness': 40,
 'Tag_health': 41,
 'Tag_history': 42,
 'Tag_humor': 43,
 'Tag_inspiration': 44,
 'Tag_investing': 45,
 'Tag_ios': 46,
 'Tag_java

In [6]:
tag_columns = df.filter(regex=r'^Tag').columns
tag_columns

Index(['Tag_ai', 'Tag_android', 'Tag_apple', 'Tag_architecture', 'Tag_art',
       'Tag_artificial-intelligence', 'Tag_big-data', 'Tag_bitcoin',
       'Tag_blacklivesmatter', 'Tag_blockchain', 'Tag_blog', 'Tag_blogging',
       'Tag_books', 'Tag_branding', 'Tag_business', 'Tag_college',
       'Tag_computer-science', 'Tag_creativity', 'Tag_cryptocurrency',
       'Tag_culture', 'Tag_data', 'Tag_data-science', 'Tag_data-visualization',
       'Tag_deep-learning', 'Tag_design', 'Tag_dogs', 'Tag_donald-trump',
       'Tag_economics', 'Tag_education', 'Tag_energy', 'Tag_entrepreneurship',
       'Tag_environment', 'Tag_ethereum', 'Tag_feminism', 'Tag_fiction',
       'Tag_food', 'Tag_football', 'Tag_google', 'Tag_government',
       'Tag_happiness', 'Tag_health', 'Tag_history', 'Tag_humor',
       'Tag_inspiration', 'Tag_investing', 'Tag_ios', 'Tag_javascript',
       'Tag_jobs', 'Tag_journalism', 'Tag_leadership', 'Tag_life',
       'Tag_life-lessons', 'Tag_love', 'Tag_machine-learning',

In [7]:
df['Tags'] = (df[tag_columns] * pd.Series(tag_weights)).sum(axis=1)
df['Tags']

1197026    115
342733      41
532172      59
578897      61
308090      36
          ... 
1072858    144
1283548    101
542523      59
534727      59
165988      23
Name: Tags, Length: 26959, dtype: int64

In [8]:
tfidf_vectorizer = TfidfVectorizer()

# Fit the TfidfVectorizer on the entire corpus
df.reset_index(drop=True, inplace=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Text'])

# Get feature names (terms)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)


def get_top_terms(row, tfidf_matrix, feature_names):
    # Get TF-IDF values for the row
    tfidf_values = tfidf_matrix[row.name].toarray()[0]
    
    # Create a DataFrame with TF-IDF values and corresponding feature names
    tfidf_df = pd.DataFrame({'Term': feature_names, 'TF-IDF': tfidf_values})
    
    # Sort the DataFrame by TF-IDF values in descending order
    tfidf_df_sorted = tfidf_df.sort_values(by='TF-IDF', ascending=False)
    
    # Get the top 3 terms and their TF-IDF values
    top_terms = tfidf_df_sorted.head(3).values.tolist()
    
    for i, elem in enumerate(top_terms):
        top_terms[i] = top_terms[i][0]
    
    return top_terms

df['Top Terms'] = df.apply(get_top_terms, args=(tfidf_matrix, feature_names), axis=1)


In [9]:
# len(df)
# df['Top Terms'] = df.apply(get_top_terms, axis=1)
# df['Top Terms'] = df.apply(get_top_terms, args=(tfidf_matrix, feature_names), axis=1)

# df['Tags_categorical'] = df['Tags'].apply(number_to_tag, axis=1)

df['Categories'] = df['Tags'].map(inverse_tag_weights)

df = df.dropna(subset=['Categories'])
print(df[['Top Terms', 'Tags', 'Categories']][df['Tags'] >= 1].head(59))


                                    Top Terms  Tags                Categories
1         [tucking, restriction, provocation]    41                Tag_health
2            [affecting, critics, definition]    59                Tag_movies
3                 [blazers, trail, defending]    61                   Tag_nba
4                      [st, chinese, takeout]    36                  Tag_food
5                    [gusts, wanderers, lips]    86                Tag_travel
6                       [car, stolen, assume]    86                Tag_travel
7            [emerging, analysis, artificial]    60                 Tag_music
8              [polarization, concern, civil]    69              Tag_politics
9                       [wish, planted, poem]    68                Tag_poetry
10                  [blindly, awoke, checked]    64             Tag_parenting
11              [ovolo, hotel, woolloomooloo]    71          Tag_productivity
12                      [10th, result, board]    29             

In [10]:
df

,Unnamed: 0,Title,Subtitle,Image,Author,Publication,Year,Month,Day,Reading_Time,...,Tag_web-design,Tag_web-development,Tag_women,Tag_wordpress,Tag_work,Tag_writing,Text,Tags,Top Terms,Categories
1,4571,Restriction: Binding and Tucking,The breasts were the provocation. The undeniab...,1,M. Whiteford,NaN,2017,9,19,6,...,0,0,0,0,0,0,Restriction: Binding and Tucking The breasts w...,41,"[tucking, restriction, provocation]",Tag_health
2,9682,A Look at the Best Films of 2017,The changing definition of film is affecting w...,1,Graham Fuller,The Omnivore,2017,12,18,6,...,0,0,0,0,0,0,A Look at the Best Films of 2017 The changing ...,59,"[affecting, critics, definition]",Tag_movies
3,2531,"Some Way, Some How, the Portland Trail Blazers...","To the surprise of everyone, the Trail Blazers...",0,Jackson Lloyd,NaN,2017,11,25,3,...,0,0,0,0,0,0,"Some Way, Some How, the Portland Trail Blazers...",61,"[blazers, trail, defending]",Tag_nba
4,21486,A Longstanding Chinese Restaurant in Harlem,"Behind the bullet proof booth, Jason Lin() too...",0,Tzu-Yun Lu,NaN,2018,3,14,3,...,0,0,0,0,0,0,A Longstanding Chinese Restaurant in Harlem Be...,36,"[st, chinese, takeout]",Tag_food
5,25960,The Wanderers,Gusts sent from giant lips,0,Epsilon,NaN,2018,3,25,1,...,0,0,0,0,0,0,The Wanderers Gusts sent from giant lips,86,"[gusts, wanderers, lips]",Tag_travel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26952,6364,Hello World in Python: TDD Approach,Most of the time we write our code then check ...,0,Ruhshan Ahmed Abir,NaN,2018,4,22,4,...,0,0,0,0,0,0,Hello World in Python: TDD Approach Most of th...,74,"[might, we, bigger]",Tag_python
26953,14679,Supportive hearts,It happened to me,0,Woke,NaN,2018,4,17,1,...,0,0,0,0,0,0,Supportive hearts It happened to me,34,"[supportive, hearts, happened]",Tag_feminism
26956,23616,Raiders of the Lost Ark as a story of GDPR com...,I was reflecting on GDPR preparations and it s...,0,lawrence serewicz,NaN,2018,5,30,3,...,0,0,0,0,0,0,Raiders of the Lost Ark as a story of GDPR com...,59,"[gdpr, raiders, preparations]",Tag_movies
26957,13185,A Fantastic Woman 2018 Mkv Movie Download,"Four a considerable length of time ago, Sebast...",0,alena richard,NaN,2018,1,31,2,...,0,0,0,0,0,0,A Fantastic Woman 2018 Mkv Movie Download Four...,59,"[bracingly, mkv, divorcee]",Tag_movies


In [11]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


# Initialize RAKE
r = Rake()
# Function to extract keywords using RAKE
def extract_keywords(text):
    # Extract keywords from the text
    r.extract_keywords_from_text(text)
    
    # Get the ranked keywords
    keywords = r.get_ranked_phrases()
    
    return keywords

[nltk_data] Downloading package stopwords to /Users/boss/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/boss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
df['Keywords'] = df['Text'].apply(extract_keywords)

df

/var/folders/1r/_g41q09n5dl_vr4brp9p_w580000gn/T/ipykernel_18330/1964619940.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Keywords'] = df['Text'].apply(extract_keywords)


,Unnamed: 0,Title,Subtitle,Image,Author,Publication,Year,Month,Day,Reading_Time,...,Tag_web-development,Tag_women,Tag_wordpress,Tag_work,Tag_writing,Text,Tags,Top Terms,Categories,Keywords
1,4571,Restriction: Binding and Tucking,The breasts were the provocation. The undeniab...,1,M. Whiteford,NaN,2017,9,19,6,...,0,0,0,0,0,Restriction: Binding and Tucking The breasts w...,41,"[tucking, restriction, provocation]",Tag_health,"[undeniable appearance, two, tucking, restrict..."
2,9682,A Look at the Best Films of 2017,The changing definition of film is affecting w...,1,Graham Fuller,The Omnivore,2017,12,18,6,...,0,0,0,0,0,A Look at the Best Films of 2017 The changing ...,59,"[affecting, critics, definition]",Tag_movies,"[critics name, changing definition, best films..."
3,2531,"Some Way, Some How, the Portland Trail Blazers...","To the surprise of everyone, the Trail Blazers...",0,Jackson Lloyd,NaN,2017,11,25,3,...,0,0,0,0,0,"Some Way, Some How, the Portland Trail Blazers...",61,"[blazers, trail, defending]",Tag_nba,"[elite defensive team, portland trail blazers,..."
4,21486,A Longstanding Chinese Restaurant in Harlem,"Behind the bullet proof booth, Jason Lin() too...",0,Tzu-Yun Lu,NaN,2018,3,14,3,...,0,0,0,0,0,A Longstanding Chinese Restaurant in Harlem Be...,36,"[st, chinese, takeout]",Tag_food,"[jason lin () took orders, undergone three dif..."
5,25960,The Wanderers,Gusts sent from giant lips,0,Epsilon,NaN,2018,3,25,1,...,0,0,0,0,0,The Wanderers Gusts sent from giant lips,86,"[gusts, wanderers, lips]",Tag_travel,"[wanderers gusts sent, giant lips]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26952,6364,Hello World in Python: TDD Approach,Most of the time we write our code then check ...,0,Ruhshan Ahmed Abir,NaN,2018,4,22,4,...,0,0,0,0,0,Hello World in Python: TDD Approach Most of th...,74,"[might, we, bigger]",Tag_python,"[things gets bigger, fear gets bigger, tdd app..."
26953,14679,Supportive hearts,It happened to me,0,Woke,NaN,2018,4,17,1,...,0,0,0,0,0,Supportive hearts It happened to me,34,"[supportive, hearts, happened]",Tag_feminism,"[supportive hearts, happened]"
26956,23616,Raiders of the Lost Ark as a story of GDPR com...,I was reflecting on GDPR preparations and it s...,0,lawrence serewicz,NaN,2018,5,30,3,...,0,0,0,0,0,Raiders of the Lost Ark as a story of GDPR com...,59,"[gdpr, raiders, preparations]",Tag_movies,"[explain gdpr preparations, gdpr preparations,..."
26957,13185,A Fantastic Woman 2018 Mkv Movie Download,"Four a considerable length of time ago, Sebast...",0,alena richard,NaN,2018,1,31,2,...,0,0,0,0,0,A Fantastic Woman 2018 Mkv Movie Download Four...,59,"[bracingly, mkv, divorcee]",Tag_movies,"[fantastic woman 2018 mkv movie download four,..."


In [13]:
df['Top Terms']

1        [tucking, restriction, provocation]
2           [affecting, critics, definition]
3                [blazers, trail, defending]
4                     [st, chinese, takeout]
5                   [gusts, wanderers, lips]
                        ...                 
26952                    [might, we, bigger]
26953         [supportive, hearts, happened]
26956          [gdpr, raiders, preparations]
26957             [bracingly, mkv, divorcee]
26958                [byersthe, joyce, 1984]
Name: Top Terms, Length: 21734, dtype: object

In [14]:
pd.set_option('display.max_columns', None)

def expand_list(row):
    row = row[:3]
    return pd.Series(row)

new_columns = df['Top Terms'].apply(expand_list)
# df.drop('Top Terms', axis=1, inplace=True)

# df.columns = ['col1_name', 'col2_name'] 

df_see = df.loc[:, ~df.columns.isin(tag_columns)]
df_see = pd.concat([df_see, new_columns], axis=1)
df_see.rename(columns={0: 'tf_idf_1', 1: 'tf_idf_2', 2: 'tf_idf_3'}, inplace=True)

new_columns = df['Keywords'].apply(expand_list)
df_see = pd.concat([df_see, new_columns], axis=1)
df_see.rename(columns={0: 'rake_1', 1: 'rake_2', 2: 'rake_3'}, inplace=True)



df_see = df_see.drop(['Image', 'Author', 'Publication', 'Year', 'Month', 'Day', 'Reading_Time', 'Claps', 'url', 'Author_url', 'Top Terms', 'Keywords'], axis=1)



df_see.head(50)

,Unnamed: 0,Title,Subtitle,Text,Tags,Categories,tf_idf_1,tf_idf_2,tf_idf_3,rake_1,rake_2,rake_3
1,4571,Restriction: Binding and Tucking,The breasts were the provocation. The undeniab...,Restriction: Binding and Tucking The breasts w...,41,Tag_health,tucking,restriction,provocation,undeniable appearance,two,tucking
2,9682,A Look at the Best Films of 2017,The changing definition of film is affecting w...,A Look at the Best Films of 2017 The changing ...,59,Tag_movies,affecting,critics,definition,critics name,changing definition,best films
3,2531,"Some Way, Some How, the Portland Trail Blazers...","To the surprise of everyone, the Trail Blazers...","Some Way, Some How, the Portland Trail Blazers...",61,Tag_nba,blazers,trail,defending,elite defensive team,portland trail blazers,trail blazers
4,21486,A Longstanding Chinese Restaurant in Harlem,"Behind the bullet proof booth, Jason Lin() too...",A Longstanding Chinese Restaurant in Harlem Be...,36,Tag_food,st,chinese,takeout,jason lin () took orders,undergone three different owners,bullet proof booth
5,25960,The Wanderers,Gusts sent from giant lips,The Wanderers Gusts sent from giant lips,86,Tag_travel,gusts,wanderers,lips,wanderers gusts sent,giant lips,NaN
6,26302,What happens when my rental car is stolen,Before making the decision to assume the respo...,What happens when my rental car is stolen Befo...,86,Tag_travel,car,stolen,assume,rental car,cheap car,stolen
7,36673,Emerging Media Industry Analysis #1,Artificial Intelligence and Machine Learning,Emerging Media Industry Analysis #1 Artificial...,60,Tag_music,emerging,analysis,artificial,emerging media industry analysis,1 artificial intelligence,machine learning
8,37190,The risk of an American Civil War is remote,Every now and then I hear people express conce...,The risk of an American Civil War is remote Ev...,69,Tag_politics,polarization,concern,civil,hear people express concern,political polarization might,american civil war
9,29709,We Planted a Wish Deep: A Nature Poem,We made a wish,We Planted a Wish Deep: A Nature Poem We made ...,68,Tag_poetry,wish,planted,poem,nature poem,wish deep,wish
10,3087,The World is Sh*t and Im Scared,"I awoke Monday morning, blindly reached for my...",The World is Sh*t and Im Scared I awoke Monday...,64,Tag_parenting,blindly,awoke,checked,awoke monday morning,im scared,checked facebook


In [15]:
import pandas as pd
import numpy as np
import spacy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the pre-trained word embeddings model
nlp = spacy.load('en_core_web_md')  # English medium model

# Example DataFrame with TF-IDF and RAKE columns
# df = ...

# Extract top words from TF-IDF and RAKE columns
df_see = df_see.dropna()

top_words = df_see[['tf_idf_1', 'tf_idf_2', 'tf_idf_3', 'rake_1', 'rake_2', 'rake_3']].values.tolist()


# print(top_words)
# Convert top words to word embeddings
word_embeddings = []
for words in top_words:
    embeddings = []
    for word in words:
        doc = nlp(word)
        if doc.has_vector:  # Check if the word has embeddings
            embeddings.append(doc.vector)
    if embeddings:  # If at least one word in the list has embeddings
        word_embeddings.append(np.mean(embeddings, axis=0))
    else:  # If none of the words have embeddings, use zero vector
        word_embeddings.append(np.zeros(nlp.vocab.vectors_length))

# Convert word embeddings to DataFrame
embedding_df = pd.DataFrame(word_embeddings)



In [17]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(embedding_df, df_see['Tags'], test_size=0.2, random_state=42)

# Initialize logistic regression model
model = LogisticRegression(C=0.01, penalty='l2')

# Train the model
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.20763243786090887


/Users/boss/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
